In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import os
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
import torch
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader

Taking a sample Input to test frequency

In [61]:
train_timestamp = "D:/NCSU/spring_2023/Neural Networks/terrain_prediction/TrainingData/subject_001_02__x_time.csv"
train_feature = "D:/NCSU/spring_2023/Neural Networks/terrain_prediction/TrainingData/subject_001_02__x.csv"
test_timestamp = "D:/NCSU/spring_2023/Neural Networks/terrain_prediction/TrainingData/subject_001_02__y_time.csv"
test_labels = "D:/NCSU/spring_2023/Neural Networks/terrain_prediction/TrainingData/subject_001_02__y.csv"

In [75]:
#X_features
x_time_df = pd.read_csv(train_timestamp, header=None)
x_time_df.astype('float64')
x_df = pd.read_csv(train_feature, header=None)
x_combined = pd.concat([x_time_df, x_df], axis=1, ignore_index=True)
x_combined = x_combined.rename({0:'timestamp'}, axis='columns')
x_combined.set_index('timestamp', inplace=True)

#Y_features
y_time_df = pd.read_csv(test_timestamp , header=None)
y_time_df.astype('float64')
y_df = pd.read_csv(test_labels, header=None)
y_combined = pd.concat([y_time_df, y_df], axis=1, ignore_index=True)
y_combined = y_combined.rename({0:'timestamp'}, axis='columns')
y_combined.set_index('timestamp', inplace=True)

Use Nearest timestamp sample to assign the label

In [77]:
combined = pd.merge_asof(left=x_combined, right=y_combined, left_index=True, right_index = True, direction='nearest')

70172

array([[ 1.726654e+00,  9.619981e+00,  1.723327e+00, -1.996891e-03,
         6.750226e-02,  0.000000e+00],
       [ 2.225759e+00,  9.493385e+00,  1.782374e+00,  8.556650e-03,
         2.933330e-02,  0.000000e+00],
       [ 2.010621e+00,  9.481603e+00,  1.770000e+00, -4.651043e-03,
         1.009082e-03,  0.000000e+00],
       ...,
       [ 4.035121e+00,  8.696340e+00,  3.040610e+00, -2.531522e-02,
         2.466749e-02,  0.000000e+00],
       [ 4.013900e+00,  8.729596e+00,  3.006278e+00, -1.408438e-02,
         2.854353e-03,  0.000000e+00],
       [ 3.631926e+00,  8.440222e+00,  3.063185e+00,  3.863595e-03,
        -2.551200e-02,  0.000000e+00]])